# Chess Engine with PyTorch

## Imports

In [3]:
import os
import numpy as np # type: ignore
import time
import torch
import torch.nn as nn # type: ignore
import torch.optim as optim # type: ignore
from torch.utils.data import DataLoader # type: ignore
from chess import pgn # type: ignore
from tqdm import tqdm # type: ignore

# Data preprocessing

## Load data

In [4]:
def load_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    return games

files = [file for file in os.listdir("../../data/pgn") if file.endswith(".pgn")]
LIMIT_OF_FILES = min(len(files), 20)
games = []
i = 1
for file in tqdm(files):
    games.extend(load_pgn(f"../../data/pgn/{file}"))
    if i >= LIMIT_OF_FILES:
        break
    i += 1

 24%|██▍       | 19/78 [00:32<01:39,  1.69s/it]


In [6]:
print(f"GAMES PARSED: {len(games)}")

GAMES PARSED: 2572


## Convert data into tensors

In [1]:
from auxiliary_func import create_input_for_nn, encode_moves

In [5]:
X, y = create_input_for_nn(games)


In [6]:
X = X[0:2500000]
y = y[0:2500000]

In [7]:
y, move_to_int = encode_moves(y)
num_classes = len(move_to_int)

In [8]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

# Preliminary actions

In [10]:
from dataset import ChessDataset
from model import ChessModel

In [11]:
# Create Dataset and DataLoader
dataset = ChessDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Model Initialization
model = ChessModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Using device: cpu


# Training

In [31]:
num_epochs = 50
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()

        outputs = model(inputs)  # Raw logits

        # Compute loss
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        running_loss += loss.item()
    end_time = time.time()
    epoch_time = end_time - start_time
    minutes: int = int(epoch_time // 60)
    seconds: int = int(epoch_time) - minutes * 60
    print(f'Epoch {epoch + 1 + 50}/{num_epochs + 1 + 50}, Loss: {running_loss / len(dataloader):.4f}, Time: {minutes}m{seconds}s')

100%|██████████| 3369/3369 [03:08<00:00, 17.87it/s]


Epoch 51/101, Loss: 0.2015, Time: 3m8s


100%|██████████| 3369/3369 [03:13<00:00, 17.44it/s]


Epoch 52/101, Loss: 0.2010, Time: 3m13s


100%|██████████| 3369/3369 [03:15<00:00, 17.20it/s]


...

Epoch 98/101, Loss: 0.1848, Time: 2m53s


100%|██████████| 3369/3369 [02:53<00:00, 19.39it/s]


Epoch 99/101, Loss: 0.1821, Time: 2m53s


100%|██████████| 3369/3369 [02:55<00:00, 19.22it/s]

Epoch 100/101, Loss: 0.1837, Time: 2m55s


# Save the model and mapping

In [12]:
# Save the model
torch.save(model.state_dict(), "../models/TORCH_100EPOCHS.pth")

In [13]:
import pickle

with open("../models/heavy_move_to_int", "wb") as file:
    pickle.dump(move_to_int, file)